<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,image_filename,mask_filename,class
0,image_003.png,mask_003.png,benign
1,image_004.png,mask_004.png,benign
2,image_008.png,mask_008.png,benign
3,image_012.png,mask_012.png,benign
4,image_016.png,mask_016.png,benign
...,...,...,...
332,image_399.png,mask_399.png,normal
333,image_400.png,mask_400.png,normal
334,image_416.png,mask_416.png,normal
335,image_419.png,mask_419.png,normal


In [8]:
# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas
dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})

dftest

,image_filename
0,image_005.png
1,image_006.png
2,image_007.png
3,image_009.png
4,image_010.png
...,...
108,image_436.png
109,image_441.png
110,image_446.png
111,image_447.png


In [11]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


In [ ]:
import tensorflow as tf
from tensorflow.data import Dataset

# Especificar información dependiente del conjunto de datos
img_width = img_height = 224  # dimensiones de la imagen
n_channels = 3                # número de canales (RGB)
n_classes = 5                 # número de clases
x_col = 'image_name'          # nombres de las columnas en el fichero CSV
y_col = 'class'

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_name, label_str):
    image_path = tf.strings.join([imgtrain_dir, image_name])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    image = tf.image.resize(image, [img_width, img_height])
    image = image / 255.0                               # normalización
    label = tf.strings.to_number(label_str, tf.int32)   # conversión de etiquetas a enteros
    label = tf.one_hot(label, depth=n_classes)          # codificación one-hot
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_paths = df[x_col].values
    labels = df[y_col].values
    dataset = Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 128
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')